In [1]:
!pip install pyspark

You should consider upgrading via the '/home/phuhien/PycharmProjects/ocvx/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
from IPython.display import display

In [3]:
spark_application_name = "Spark_Application_Name"

In [4]:
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/15 02:23:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/15 02:23:55 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/15 02:23:55 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/15 02:23:55 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/15 02:23:55 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/06/15 02:23:55 WARN 

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Cannot assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:459)
	at java.base/sun.nio.ch.Net.bind(Net.java:448)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:227)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContex

In [5]:
filePath = "sf-stocks-clean.parquet"
stocksDF = spark.read.parquet(filePath)
display(stocksDF)

NameError: name 'spark' is not defined

## Train/Test Split

In [6]:
trainDF, testDF = stocksDF.randomSplit([.8, .2], seed=42)
print(f"There are {trainDF.cache().count()} rows in the training set, and {testDF.cache().count()} in the test set")

There are 5102 rows in the training set, and 1231 in the test set


In [8]:
trainDF.select("Low","Volume").summary().show()
# DK WHAT TO DO HERE ?

+-------+------------------+-------------------+
|summary|              High|             Volume|
+-------+------------------+-------------------+
|  count|              5102|               5102|
|   mean| 536.2721293407158|3.637840187612701E7|
| stddev| 699.8217961274913|5.058271778480273E7|
|    min|29.127500534057617|           347500.0|
|    25%| 70.80000305175781|          3737600.0|
|    50%| 164.6300048828125|          1.88807E7|
|    75%|             995.5|           4.0798E7|
|    max|           3552.25|           4.4794E8|
+-------+------------------+-------------------+



## Vector Assembler

In [9]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["Volume"], outputCol="features")

vecTrainDF = vecAssembler.transform(trainDF)

vecTrainDF.select("Volume", "features", "Low").show(10)

# We combine the column Volume and Low into 1 single vector column called "features"

+---------+-----------+------------------+
|   Volume|   features|              High|
+---------+-----------+------------------+
|2.96165E7|[2.96165E7]|44.066001892089844|
|5.60675E7|[5.60675E7]|45.599998474121094|
|2.76395E7|[2.76395E7]|  46.0620002746582|
|1.98975E7|[1.98975E7]| 46.38399887084961|
|   1.83E7|   [1.83E7]|46.400001525878906|
| 1.8951E7| [1.8951E7]| 46.13999938964844|
|2.30875E7|[2.30875E7]|47.992000579833984|
| 1.8845E7| [1.8845E7]| 47.94200134277344|
|3.86615E7|[3.86615E7]|49.736000061035156|
|2.10215E7|[2.10215E7]| 49.20000076293945|
+---------+-----------+------------------+
only showing top 10 rows



## Linear Regression

In [19]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="Low", regParam=0.01)
lrModel = lr.fit(vecTrainDF)

In [31]:
m = round(lrModel.coefficients[0], 2)
b = round(lrModel.intercept, 2)

print(f"The formula for the linear regression line is Low = {m}*Volume + {b}")

The formula for the linear regression line is High = -0.0*Volume + 743.05


## Pipeline

In [21]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)

## Apply to Test Set

In [32]:
predDF = pipelineModel.transform(testDF)

predDF.select("Volume", "features", "Low", "prediction").show(10)

+---------+-----------+------------------+-----------------+
|   Volume|   features|              High|       prediction|
+---------+-----------+------------------+-----------------+
|2.95585E7|[2.95585E7]| 45.49599838256836|575.0362520479239|
| 1.8254E7| [1.8254E7]| 45.99599838256836|639.2906992676967|
| 3.0465E7| [3.0465E7]| 47.56999969482422|569.8837325729604|
|3.13145E7|[3.13145E7]|50.178001403808594|565.0551994301238|
|2.05805E7|[2.05805E7]|51.178001403808594|626.0669413432594|
|1.78125E7|[1.78125E7]| 51.56399917602539|641.8001723489944|
| 3.6726E7| [3.6726E7]| 57.47800064086914|534.2963917756675|
| 7.4576E7| [7.4576E7]| 52.93199920654297|319.1580993020153|
|1.93965E7|[1.93965E7]|49.731998443603516|632.7967626983136|
|1.53315E7|[1.53315E7]| 49.29999923706055|655.9021037526174|
+---------+-----------+------------------+-----------------+
only showing top 10 rows



## Baseline Model

In [35]:
from pyspark.ml.evaluation import RegressionEvaluator

regressionMeanEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Low", metricName="rmse")

print(f"The RMSE for predicting the Low is: {regressionMeanEvaluator.evaluate(predDF):.2f}")


The RMSE for predicting the High is: 651.36
